# Clean Price Data

In [27]:
from datetime import datetime
import os
import pandas as pd
import numpy as np

# Get Dates

In [28]:
def get_dates(root_folder):
    files = os.listdir(root_folder)
    return sorted([datetime.strptime(file.split("_")[1].split(".")[0], '%d-%m-%Y') 
            for file in files if ".csv" in file])

In [29]:
root_folder = "raw_data"
dates = get_dates(root_folder)

# Get Data

In [30]:
def get_data(dates):
    data_list = [pd.read_csv("raw_data/data_{0}.csv".format(date.strftime('%d-%m-%Y'))) for date in dates]
    return pd.concat(data_list)

In [31]:
final_data = get_data(dates)

In [32]:
final_data.head()

,Producto,Categoría,Anterior,Último,Diferencia,Medida,Tendencia,Fecha
0,Cereales,Trigo panificable,"0,00 / 0,00","147,00","147,00",Eur / Tm.,NaN,02-01-2006
1,Cereales,Cebada,"0,00 / 0,00","145,00","145,00",Eur / Tm.,NaN,02-01-2006
2,Cereales,Avena,"0,00 / 0,00","145,00","145,00",Eur / Tm.,NaN,02-01-2006
3,Cereales,Centeno,"0,00 / 0,00","139,00","139,00",Eur / Tm.,NaN,02-01-2006
4,Cereales,Maíz,"0,00 / 0,00","142,00","142,00",Eur / Tm.,NaN,02-01-2006


In [33]:
final_data[final_data["Categoría"]=="Cebada"].head()

,Producto,Categoría,Anterior,Último,Diferencia,Medida,Tendencia,Fecha
1,Cereales,Cebada,"0,00 / 0,00","145,00","145,00",Eur / Tm.,NaN,02-01-2006
1,Cereales,Cebada,"145,00","145,00","0,00",Eur / Tm.,NaN,09-01-2006
1,Cereales,Cebada,"145,00","145,00","0,00",Eur / Tm.,NaN,16-01-2006
1,Cereales,Cebada,"145,00","145,00","0,00",Eur / Tm.,NaN,23-01-2006
1,Cereales,Cebada,"145,00","144,00","-1,00",Eur / Tm.,NaN,30-01-2006


In [34]:
final_data[final_data["Categoría"]=="Cebada"].tail()

,Producto,Categoría,Anterior,Último,Diferencia,Medida,Tendencia,Fecha
1,Cereales,Cebada,"179,00 / 0,00","182,00 / 0,00","3,00",Eur / Tm.,NaN,29-10-2018
1,Cereales,Cebada,"182,00 / 0,00","184,00 / 0,00","2,00",Eur / Tm.,NaN,05-11-2018
1,Cereales,Cebada,"184,00 / 0,00","186,00 / 0,00","2,00",Eur / Tm.,NaN,12-11-2018
1,Cereales,Cebada,"186,00 / 0,00","188,00 / 0,00","2,00",Eur / Tm.,NaN,19-11-2018
1,Cereales,Cebada,"188,00 / 0,00","190,00 / 0,00","2,00",Eur / Tm.,NaN,26-11-2018


# Clean Data

In [35]:
def get_price(row, row_name):
    value = float(row[row_name].split("/")[0].strip().replace(",", "."))
    if value == 0.0:
        return np.nan
    else:
        return value

In [36]:
final_data["Anterior"] = final_data.apply(lambda row: get_price(row, "Anterior"), axis=1)

In [37]:
final_data["Último"] = final_data.apply(lambda row: get_price(row, "Último"), axis=1)

In [38]:
final_data["Diferencia"] = final_data.apply(lambda row: get_price(row, "Diferencia"), axis=1)

In [39]:
final_data[final_data["Categoría"]=="Cebada"].tail()

,Producto,Categoría,Anterior,Último,Diferencia,Medida,Tendencia,Fecha
1,Cereales,Cebada,179.0,182.0,3.0,Eur / Tm.,NaN,29-10-2018
1,Cereales,Cebada,182.0,184.0,2.0,Eur / Tm.,NaN,05-11-2018
1,Cereales,Cebada,184.0,186.0,2.0,Eur / Tm.,NaN,12-11-2018
1,Cereales,Cebada,186.0,188.0,2.0,Eur / Tm.,NaN,19-11-2018
1,Cereales,Cebada,188.0,190.0,2.0,Eur / Tm.,NaN,26-11-2018


# Get Grouped Data

In [40]:
dict_df = {key:df for key, df in final_data.groupby("Categoría")}

# Impute Missing Data

In [41]:
for key in dict_df.keys():
    dict_df[key]["Anterior"] = dict_df[key]["Anterior"].interpolate(axis=0)
    dict_df[key]["Último"] = dict_df[key]["Último"].interpolate(axis=0)

# Export Weekly Data

In [42]:
writer_weekly = pd.ExcelWriter('weekly_data.xlsx', engine='xlsxwriter')

In [43]:
for key in dict_df.keys():
    sheet_name = key[:30]
    chars = '[]:*?/\"'
    for c in chars:
        sheet_name = sheet_name.replace(c, "")
    print(sheet_name)
    dict_df[key].to_excel(writer_weekly, sheet_name=sheet_name)

15,01 a 19 KG.
Fabula, Marfona
Trigo pienso
Trigo Blando p.e. > 72 kgHl
Girasol A.O.
Lenteja armuña D.O.
Guisantes
Especial mayor 350 Kg
Costillas (Peladas)
13,01 a 15 KG.
Graso
25,5 a 28 KG.
Marranos 30 - 60 Kg
Centeno
Recebo
Tapilla
Extra mayor 270 Kg
Cebada
Girasol
Segunda menor 270 Kg
Fábrica
Paja paquete grande
Extra macho
D. O. Guijuelo
6,01 A 8 Kg
Primera menor 270 Kg
Lagarto
Normal
segunda mayor  270 Kg
Tocino de Lomo
Segunda hembra
Especial mayor 270 Kg
Especial hembra  de 180 a 220 
Primera
Marranos ibéricos puros
Solomillo
Primera hembra   de 180 a 220 
Bellota 50 % IB
Trigo panificable
Reproductoras > de 6 meses
Bellota 100 % IB
Especial menor 350 Kg
Marranos ibéricos cruzados
Lenteja de Salamanca
Bellota 75% IB
Hasta 11 Kg
Forrajes (Avena)
Barriguera
Presa Bellota
Red Pontiac
28,01 a 34 KG.
Segunda hembra  de 180 a 220 K
Pienso 150 Kg
Lomo Bellota
Colza
Presa
Cebo
Segunda mayor 350 Kg
Reproductoras
Lomo Cebo
Tercera
Tostones 5 - 7 Kg
Selecto
Primera menor 350 Kg
Paja paque

In [44]:
writer_weekly.save()

In [45]:
[item for item in list(dict_df.keys()) if "Tri" in item]

['Trigo pienso',
 'Trigo Blando p.e. > 72 kg/Hl',
 'Trigo panificable',
 'Triticale']

In [46]:
dict_df["Trigo Blando p.e. > 72 kg/Hl"].tail()

,Producto,Categoría,Anterior,Último,Diferencia,Medida,Tendencia,Fecha
0,Cereales,Trigo Blando p.e. > 72 kg/Hl,186.0,187.0,1.0,Eur / Tm.,NaN,29-10-2018
0,Cereales,Trigo Blando p.e. > 72 kg/Hl,187.0,188.0,1.0,Eur / Tm.,NaN,05-11-2018
0,Cereales,Trigo Blando p.e. > 72 kg/Hl,188.0,190.0,2.0,Eur / Tm.,NaN,12-11-2018
0,Cereales,Trigo Blando p.e. > 72 kg/Hl,190.0,192.0,2.0,Eur / Tm.,NaN,19-11-2018
0,Cereales,Trigo Blando p.e. > 72 kg/Hl,192.0,193.0,1.0,Eur / Tm.,NaN,26-11-2018


# Group & Export Monthly Data

In [47]:
dict_df["Cebada"].head()

,Producto,Categoría,Anterior,Último,Diferencia,Medida,Tendencia,Fecha
1,Cereales,Cebada,NaN,145.0,145.0,Eur / Tm.,NaN,02-01-2006
1,Cereales,Cebada,145.0,145.0,NaN,Eur / Tm.,NaN,09-01-2006
1,Cereales,Cebada,145.0,145.0,NaN,Eur / Tm.,NaN,16-01-2006
1,Cereales,Cebada,145.0,145.0,NaN,Eur / Tm.,NaN,23-01-2006
1,Cereales,Cebada,145.0,144.0,-1.0,Eur / Tm.,NaN,30-01-2006


In [48]:
def get_year_month(row):
    date = datetime.strptime(row["Fecha"], '%d-%m-%Y')
    return date.replace(day=1)

In [49]:
def format_year_month(row):
    return row.strftime('%m-%Y')

In [50]:
writer_monthly = pd.ExcelWriter('monthly_data.xlsx', engine='xlsxwriter')

In [51]:
dict_df_montly = {}
for key in dict_df.keys():
    sheet_name = key[:30]
    chars = '[]:*?/\"'
    for c in chars:
        sheet_name = sheet_name.replace(c, "")
    print(sheet_name)
    month_data = dict_df[key]
    month_data["Fecha-Sort"] = month_data.apply(lambda row: get_year_month(row), axis=1)
    grouped_data = month_data.groupby(["Producto", "Categoría", "Fecha-Sort", "Medida"])["Último"].mean().reset_index()
    grouped_data["Fecha"] = grouped_data["Fecha-Sort"].apply(lambda row: format_year_month(row))
    grouped_data = grouped_data.drop(["Fecha-Sort"], axis=1)
    grouped_data.to_excel(writer_monthly, sheet_name=sheet_name)

15,01 a 19 KG.
Fabula, Marfona
Trigo pienso
Trigo Blando p.e. > 72 kgHl
Girasol A.O.
Lenteja armuña D.O.
Guisantes
Especial mayor 350 Kg
Costillas (Peladas)
13,01 a 15 KG.
Graso
25,5 a 28 KG.
Marranos 30 - 60 Kg
Centeno
Recebo
Tapilla
Extra mayor 270 Kg
Cebada
Girasol
Segunda menor 270 Kg
Fábrica
Paja paquete grande
Extra macho
D. O. Guijuelo
6,01 A 8 Kg
Primera menor 270 Kg
Lagarto
Normal
segunda mayor  270 Kg
Tocino de Lomo
Segunda hembra
Especial mayor 270 Kg
Especial hembra  de 180 a 220 
Primera
Marranos ibéricos puros
Solomillo
Primera hembra   de 180 a 220 
Bellota 50 % IB
Trigo panificable
Reproductoras > de 6 meses
Bellota 100 % IB
Especial menor 350 Kg
Marranos ibéricos cruzados
Lenteja de Salamanca
Bellota 75% IB
Hasta 11 Kg
Forrajes (Avena)
Barriguera
Presa Bellota
Red Pontiac
28,01 a 34 KG.
Segunda hembra  de 180 a 220 K
Pienso 150 Kg
Lomo Bellota
Colza
Presa
Cebo
Segunda mayor 350 Kg
Reproductoras
Lomo Cebo
Tercera
Tostones 5 - 7 Kg
Selecto
Primera menor 350 Kg
Paja paque

In [52]:
writer_monthly.save()